# Presentazione del progetto

## Titolo
Pipeline avanzata per classificare i menu e orchestrare un RAG ibrido specializzato.
**Differenza vs simple_rag:** la presentazione introduce da subito la natura adattiva del flusso invece del solo estrattore unico.

## Obiettivo
Portare la pipeline base verso un flusso piu robusto e osservabile che unisce classificazione, estrazione multimodale e agenti tool-based.
- classificare automaticamente ogni documento in structured o unstructured prima dell'estrazione;
- riutilizzare ingredienti e tecniche dai menu puliti per guidare l'analisi dei menu rumorosi;
- abilitare interrogazioni semantiche complesse tramite un agente con strumenti di lookup e set-ops;
- mantenere la valutazione automatica per confrontare le prestazioni con il ground truth.
**Differenza vs simple_rag:** l'obiettivo include classificazione preventiva e estrazione guidata, assenti nell'approccio precedente.

## Dati
- Cartella principale: Dataset (immutata rispetto a simple_rag).
- Knowledge_base/menu: sorgente dei PDF da classificare ed estrarre.
- ground_truth: mapping ufficiali e domande per la valutazione.
- artifacts/advanced_rag: contiene i nuovi output (`menu_classifications.json`, `extracted_menu_info.json`, mapping aggiornati, log valutativi).
**Differenza vs simple_rag:** il set di artifact cresce includendo le etichette di classificazione e gli output separati per menu non strutturati.

## Architettura / Pipeline
1. Parsing e aggregazione condivisi
     - parse_documents_in_directory() + group_and_concatenate_documents().
     - output: `parsed_menus.json`.
     - Differenza vs simple_rag: stesso step, ma diventa l'input comune per la classificazione successiva.

2. Classificazione dei menu
     - classify_menu(..., model_name="gpt-4.1-mini").
     - etichetta ogni documento come structured/unstructured e salva `menu_classifications.json`.
     - Differenza vs simple_rag: nuovo gate decisionale per scegliere il percorso di estrazione.

3. Estrazione adattiva
     - extract_info_from_menus(..., classifications=...).
     - i menu strutturati usano l'estrattore standard; ingredienti e tecniche ricavati vengono riusati per un prompt mirato sui menu non strutturati.
     - Differenza vs simple_rag: doppio pass estrattivo con knowledge transfer verso gli input rumorosi.

4. Creazione dei mapping
     - create_mappings(...) + write_json per `ingredient_to_dishes.json` e `technique_to_dishes.json`.
     - Differenza vs simple_rag: stesso obiettivo, ma alimentato da dati piu completi grazie al recupero ibrido.

5. Agente conversazionale con toolset
     - get_agent(model_name="gpt-4.1") espone tool di lookup, intersect e subtract sugli ID piatto.
     - query_dish_ids(...) consente domande multi-condizione e ritorna gli ID coerenti.
     - Differenza vs simple_rag: stessa interfaccia, ma beneficia di mapping piu ricchi e include esplicita gestione delle operazioni insiemistiche.

6. Valutazione automatica
     - evaluate_easy_questions(...) confronta le risposte con il ground truth, salvando `easy_questions_evaluation_results.csv`.
     - Differenza vs simple_rag: stessa metrica ma con aspettativa di accuratezza superiore grazie ai passi aggiuntivi.

## Risultati prodotti
- Classificazioni per ogni menu e dizionari strutturati omogenei.
- Lista consolidata di ingredienti/tecniche provenienti dai menu piu puliti.
- Mapping ingredienti/tecniche -> piatti arricchiti e normalizzati.
- Accuratezza sulle domande easy: 100%

## Come usare il notebook
- Eseguire il setup e il parsing esattamente come in simple_rag.
- Lanciare la cella di classify_menu prima di qualsiasi estrazione.
- Eseguire extract_info_from_menus per popolare sia i menu strutturati sia quelli non strutturati (verificare gli artifact).
- Rigenerare i mapping e sperimentare con query_dish_ids variando le condizioni.
- Concludere con evaluate_easy_questions per confrontare i risultati con la baseline.
**Differenza vs simple_rag:** l'ordine di esecuzione richiede la classificazione e l'estrazione adattiva, non presente nel notebook semplice.

## Limiti e sviluppi futuri
- Costi di classificazione aggiuntivi: valutare batching e caching.
- Trasferimento di ingredienti/tecniche ancora mono direzionale: possibile iterare con self-refinement.
- LLM utilizzato in piu punti: utile introdurre controlli di qualita automatizzati sugli output intermedi.
- Reranking delle risposte agente ancora basato su heuristics: considerare scoring su embedding.
**Differenza vs simple_rag:** nuovi colli di bottiglia emergono proprio dalle fasi extra di classificazione e fusione.

## Conclusione
Il notebook advanced_rag eredita la pipeline originale ma la rende piu consapevole del tipo di documento e delle esigenze di interrogazione, ottenendo una knowledge base piu completa e pronta per domande articolate rispetto al vecchio approccio.


In [ ]:
from pathlib import Path
import sys


	
cwd = Path.cwd().resolve()
project_dir = cwd.parent.parent

if str(project_dir) not in sys.path:
	sys.path.insert(0, str(project_dir))
	

dataset_file_path = project_dir / "Dataset"
artifacts_file_path = cwd / "artifacts" / "advanced_rag"

In [2]:
from src.preprocessing.menu_ingestion import group_and_concatenate_documents, parse_documents_in_directory
from src.utils import write_json


menus_path = dataset_file_path / "Knowledge_base" / "menu"
documents_pages = parse_documents_in_directory(document_path=menus_path)
documents = group_and_concatenate_documents(documents=documents_pages)
write_json(documents, artifacts_file_path / "parsed_menus.json")

for doc_name, doc_text in list(documents.items())[:5]:
    print(f"Document: {doc_name}\nContent Preview: {doc_text[:100]}...\n")

2025-12-04 01:40:01,688 - WARNING - incorrect startxref pointer(1)
2025-12-04 01:40:01,688 - WARNING - parsing for Object Streams
2025-12-04 01:40:05,047 - WARNING - incorrect startxref pointer(1)
2025-12-04 01:40:05,055 - WARNING - parsing for Object Streams
2025-12-04 01:40:06,257 - WARNING - incorrect startxref pointer(1)
2025-12-04 01:40:06,257 - WARNING - parsing for Object Streams


Document: Anima Cosmica.pdf
Content Preview: Ristorante "Anima Cosmica"
Chef Aurora Stellaris
Nel cuore pulsante di Pandora, dove le foreste biol...

Document: Armonia Universale.pdf
Content Preview: Ristorante "Armonia Universale"
Chef: Maestro Alessandro Stellanova
Su Pandora, tra le bioluminescen...

Document: Cosmica Essenza.pdf
Content Preview: Cosmica Essenza
Alla guida dello Chef Aurelio "Starweaver" Celestini su Tatooine
Nel vasto deserto d...

Document: Datapizza.pdf
Content Preview: Ristorante "L'Infinito Sapore"
Viaggio nel Tempo e nel Gusto su Pandora
Chef Alessandro-Pierpaolo-Ja...

Document: Eco di Pandora.pdf
Content Preview: Ristorante "L'Eco di Pandora"
Chef Alessandra Novastella
Nel cuore pulsante di Pandora, dove la natu...



In [3]:
from src.preprocessing.menu_classification import classify_menu

classifications = classify_menu(text_extracted=documents)
write_json(classifications, artifacts_file_path / "menu_classifications.json")

for doc_name, classification in list(classifications.items())[:5]:
    print(f"Document: {doc_name}\nClassification: {classification}\n")

2025-12-04 01:40:21,126 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-12-04 01:40:25,735 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-12-04 01:40:28,083 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-12-04 01:40:30,548 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-12-04 01:40:34,863 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-12-04 01:40:39,248 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-12-04 01:40:41,708 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-12-04 01:40:44,163 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-12-04 01:40:46,513 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-12-04 01:40:50,847 - INFO - HTTP Request:

Document: Anima Cosmica.pdf
Classification: structured

Document: Armonia Universale.pdf
Classification: structured

Document: Cosmica Essenza.pdf
Classification: structured

Document: Datapizza.pdf
Classification: unstructured

Document: Eco di Pandora.pdf
Classification: structured



In [4]:
from src.preprocessing.menu_extraction import extract_info_from_menus

extracted_info = extract_info_from_menus(documents=documents, classifications=classifications)
write_json(extracted_info, artifacts_file_path / "extracted_menu_info.json")

for info in extracted_info[:5]:
    restaurant_name = info.get("restaurant_name", "Unknown")
    print(f"Document: {restaurant_name}\nExtracted Info: {info}\n")

Extracting info from structured menus...


2025-12-04 01:42:05,846 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


Info from document: Anima Cosmica.pdf has been extracted.


2025-12-04 01:42:17,241 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


Info from document: Armonia Universale.pdf has been extracted.


...

Info from document: Ristorante Quantico.pdf has been extracted.


2025-12-04 01:49:54,119 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


Info from document: Stelle Astrofisiche.pdf has been extracted.


2025-12-04 01:50:10,683 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


Info from document: Universo Gastronomico di Namecc.pdf has been extracted.
Document: Anima Cosmica
Extracted Info: {'restaurant_name': 'Anima Cosmica', 'chef_name': 'Aurora Stellaris', 'skills': [{'skill_name': 'Psionica', 'skill_level': 3}, {'skill_name': 'Temporale', 'skill_level': 1}, {'skill_name': 'Gravitazionale', 'skill_level': 1}, {'skill_name': 'Antimateria', 'skill_level': 1}, {'skill_name': 'Quantistica', 'skill_level': 16}, {'skill_name': 'Luce', 'skill_level': 2}, {'skill_name': 'Gradi di influenza di livello tecnologico', 'skill_level': 2}], 'dishes': [{'dish_name': 'Nebulosa Celestiale alla Stellaris', 'ingredients': ['Shard di Materia Oscura', 'Carne di Balena spaziale', 'Carne di Mucca', 'Teste di Idra', 'Riso di Cassandra', 'Biscotti della Galassia', 'Pane di Luce', 'Funghi dell’Etere', 'Spezie Melange'], 'techniques': ['Cottura a Vapore con Flusso di Particelle Isoarmoniche', 'Cottura a Vapore Termocinetica Multipla', 'Taglio a Risonanza Sonica Rigenerativa', 'Cottu

In [5]:
from src.preprocessing.menu_mapping import create_mappings
from src.utils import read_json

dish_mapping = read_json(dataset_file_path / "ground_truth" / "dish_mapping.json")
ingredient_to_dishes, technique_to_dishes = create_mappings(extracted_info=extracted_info, dish_mapping=dish_mapping)

write_json(technique_to_dishes, artifacts_file_path / 'technique_to_dishes.json')
write_json(ingredient_to_dishes, artifacts_file_path / 'ingredient_to_dishes.json')

Created mappings:
- Techniques: 73 unique techniques
- Ingredients: 70 unique ingredients


In [6]:
from src.ai.agents.engine import get_agent, query_dish_ids

agent = get_agent(model_name="gpt-4.1")
response = query_dish_ids(question="Quali sono i piatti che includono le Chocobo Wings come ingrediente?", agent=agent)
print(response)

MAPPINGS_DIR is set to: C:\Users\g.liturri\Desktop\data-pizza\test-tecnico-ai-engineer\src\experiments\artifacts


2025-12-04 01:51:13 <assistant> STARTING AGENT 

2025-12-04 01:51:13 <assistant> --- STEP 1 --- 

2025-12-04 01:51:14,025 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


<assistant>
╭────────────────────────────────────── TOOL GET_INGREDIENT_DISH_IDS RESULT ──────────────────────────────────────╮
│ {"ingredient": "Chocobo Wings", "dish_ids": [78]}                                                               │
╰─ args: {'ingredient': 'Chocobo Wings'} ─────────────────────────────────────────────────────────────────────────╯

2025-12-04 01:51:14 <assistant> --- STEP 2 --- 

2025-12-04 01:51:14,840 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


<assistant>
╭───────────────────────────────────────────────── FINAL ANSWER ──────────────────────────────────────────────────╮
│ ```json                                                                                                         │
│ [78]                                                                                                            │
│ ```                                                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

{78}


In [ ]:
from src.evaluation.easy_questions_evaluation import evaluate_easy_questions


question_path = dataset_file_path / "domande.csv"
ground_truth_path = dataset_file_path / "ground_truth" / "ground_truth_mapped.csv"
eval_df = evaluate_easy_questions(agent=agent, question_path=question_path, ground_truth_path=ground_truth_path)
eval_df.to_csv(artifacts_file_path / "easy_questions_evaluation_results.csv", index=False)

2025-12-04 01:51:26 <assistant> STARTING AGENT 

2025-12-04 01:51:26 <assistant> --- STEP 1 --- 

2025-12-04 01:51:27,325 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


<assistant>
╭────────────────────────────────────── TOOL GET_INGREDIENT_DISH_IDS RESULT ──────────────────────────────────────╮
│ {"ingredient": "Chocobo Wings", "dish_ids": [78]}                                                               │
╰─ args: {'ingredient': 'Chocobo Wings'} ─────────────────────────────────────────────────────────────────────────╯

2025-12-04 01:51:27 <assistant> --- STEP 2 --- 

...

2025-12-04 01:54:56 <assistant> --- STEP 4 --- 

2025-12-04 01:54:57,353 - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


<assistant>
╭───────────────────────────────────────────────── FINAL ANSWER ──────────────────────────────────────────────────╮
│ ```json                                                                                                         │
│ [169]                                                                                                           │
│ ```                                                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

  atteso:    [169]
  predetto:  [169]
[048] 1.00 - Quali piatti combinano la potenza dello Shard di Materia Oscura con le misteriose Uova di Fenice, utilizzando sia l'Affumicatura Polarizzata a Freddo Iperbarico che la Cottura a Vapore Ecodinamico Bilanciato?
--------------------


Accuratezza Easy: (100.00%)


TypeError: Index(...) must be called with a collection of some kind, False was passed

In [ ]:
print("Evaluation Results:")
print(eval_df)
print("Accuracy:", eval_df['correct'].mean())